## Shelby County Transportation Recovery Model

In [1]:
import json
import pandas as pd
from pyincore import InsecureIncoreClient, InventoryDataset
from pyincore.analyses.bridgedamage import BridgeDamage
from pyincore.analyses.transportationrecovery import TransportationRecovery
from pyincore.analyses.transportationrecovery import TransportationRecoveryUtil

## RUN Bridge damage analysis

### Set up input datasets and parameters

In [2]:
client = InsecureIncoreClient("http://incore2-services.ncsa.illinois.edu:8888", "incrtest")
hazard_type = "earthquake"
hazard_id = "5b902cb273c3371e1236b36b"
dmg_ratio_id = "5a284f2cc7d30d13bc081f96"

# NBSR bridges
bridge_dataset_id = "5a284f2dc7d30d13bc082040"

# Default Bridge Fragility Mapping on incore-service
mapping_id = "5b47bcce337d4a37755e0cb2"

# Use hazard uncertainty for computing damage
use_hazard_uncertainty = False

# Use liquefaction (LIQ) column of bridges to modify fragility curve
use_liquefaction = False

In [3]:
# Create bridge damage 
bridge_dmg = BridgeDamage(client)

# Load input datasets
bridge_dmg.load_remote_input_dataset("bridges", bridge_dataset_id)
bridge_dmg.load_remote_input_dataset("dmg_ratios", dmg_ratio_id)

# Specify the result name
result_name = "bridge_result"

# Set analysis parameters
bridge_dmg.set_parameter("result_name", result_name)
bridge_dmg.set_parameter("mapping_id", mapping_id)
bridge_dmg.set_parameter("hazard_type", hazard_type)
bridge_dmg.set_parameter("hazard_id", hazard_id)
bridge_dmg.set_parameter("num_cpu", 4)

It already exsists; no unzip
It already exsists; no unzip


True

In [4]:
bridge_dmg.run_analysis()

True

### damamge result ouptut

In [5]:
bridge_damage = pd.read_csv('bridge_result.csv')
bridge_damage.head()

,guid,ls-slight,ls-moderat,ls-extensi,ls-complet,none,slight-mod,mod-extens,ext-comple,complete,meandamage,expectval,retrofit,retro_cost,hazardtype,hazardval
0,dd323e98-811b-4e04-9421-608759833360,0.740530,0.160071,0.064320,0.022382,0.259470,0.580459,0.095751,0.041938,0.022382,0.057940,Moderate,as built,0.0,pga,0.314129
1,f6d5befe-88c1-4833-9164-6d7ba2b66ca6,0.741328,0.160672,0.064631,0.022513,0.258672,0.580657,0.096041,0.042117,0.022513,0.058146,Moderate,as built,0.0,pga,0.314672
2,033553d8-100b-4c0f-87a6-de74ffbee25c,0.757964,0.173768,0.071501,0.025464,0.242036,0.584196,0.102267,0.046037,0.025464,0.062680,Moderate,as built,0.0,pga,0.326410
3,b7312b14-76fc-4548-84aa-f331885a41d1,0.858277,0.261609,0.128316,0.059430,0.141723,0.596668,0.133293,0.068886,0.059430,0.105215,Moderate,as built,0.0,pga,0.339001
4,103283a3-548e-4368-a38a-bee6fae481ed,0.854595,0.256344,0.124937,0.057531,0.145405,0.598251,0.131406,0.067406,0.057531,0.102843,Moderate,as built,0.0,pga,0.335163


## lookup average daily traffic from National Bridge Inventory

In [6]:
bridges_set = InventoryDataset('cache_data/nbsrbridges/nbsrbridges.shp')
bridges = list(bridges_set.inventory_set)
ADT = TransportationRecoveryUtil.get_average_daily_traffic(bridges, 'Data/TN17.shp')

In [7]:
ADT

{'dd323e98-811b-4e04-9421-608759833360': 22440,
 'f6d5befe-88c1-4833-9164-6d7ba2b66ca6': 70820,
 '033553d8-100b-4c0f-87a6-de74ffbee25c': 116550,
 'b7312b14-76fc-4548-84aa-f331885a41d1': 97890,
 '103283a3-548e-4368-a38a-bee6fae481ed': 89430,
 'fb960495-a653-47a7-890f-de3a62e2ae2d': 20540,
 'cb175bb2-03ef-4814-ab22-ba7fd1dd5c91': 129470,
 'b39f942a-555a-4677-9b62-b13441d6f503': 14130,
 '66fd8603-c038-4a4d-aec2-667b0ec0e948': 6300,
 '72022b56-b4d0-4708-b44a-434d76490362': 83630,
 '165914a0-a651-4b13-8dfa-841247935db0': 65460,
 '3d84ef35-6170-4386-a18d-43100983345d': 73900,
 '2bcc4558-e8be-4cd8-9259-6b4872286783': 73900,
 '8e9a135d-339a-4cb0-87e6-a75589a39823': 103430,
 'f82b2aa2-eb61-4f8d-b98f-af4eb5cae488': 39110,
 'd5ae7be5-b74f-4f56-92bc-8b246a49fd8f': 29880,
 '69c93b3e-dc58-412f-bc7e-6c6363b44ccf': 16870,
 '66a1b90e-cac6-41ff-91d7-16953d9c3e61': 12160,
 '1e0e2f70-69ee-41a1-bb45-a41dc8194921': 17890,
 '6d48f606-8e8e-43be-bbc8-7be548f06505': 7770,
 '98b8083e-b88f-48a8-88b1-045bc5e26d36'

### convert bridge damage output to the format that transportation recovery model accepts

In [8]:
bridge_damage_value, unrepaired_bridge = TransportationRecoveryUtil.convert_dmg_prob2state("bridge_result.csv")

In [9]:
bridge_damage_value

{'dd323e98-811b-4e04-9421-608759833360': 1,
 'f6d5befe-88c1-4833-9164-6d7ba2b66ca6': 1,
 '033553d8-100b-4c0f-87a6-de74ffbee25c': 1,
 'b7312b14-76fc-4548-84aa-f331885a41d1': 1,
 '103283a3-548e-4368-a38a-bee6fae481ed': 1,
 'fb960495-a653-47a7-890f-de3a62e2ae2d': 1,
 'cb175bb2-03ef-4814-ab22-ba7fd1dd5c91': 1,
 'b39f942a-555a-4677-9b62-b13441d6f503': 1,
 '66fd8603-c038-4a4d-aec2-667b0ec0e948': 1,
 '72022b56-b4d0-4708-b44a-434d76490362': 1,
 '165914a0-a651-4b13-8dfa-841247935db0': 1,
 '3d84ef35-6170-4386-a18d-43100983345d': 1,
 '2bcc4558-e8be-4cd8-9259-6b4872286783': 1,
 '8e9a135d-339a-4cb0-87e6-a75589a39823': 1,
 'f82b2aa2-eb61-4f8d-b98f-af4eb5cae488': 1,
 'd5ae7be5-b74f-4f56-92bc-8b246a49fd8f': 1,
 '69c93b3e-dc58-412f-bc7e-6c6363b44ccf': 1,
 '66a1b90e-cac6-41ff-91d7-16953d9c3e61': 1,
 '1e0e2f70-69ee-41a1-bb45-a41dc8194921': 2,
 '6d48f606-8e8e-43be-bbc8-7be548f06505': 1,
 '98b8083e-b88f-48a8-88b1-045bc5e26d36': 1,
 '9859f11b-e11a-4fc0-a96e-e5989a6a244c': 1,
 '4c8792c5-70a8-4c1c-b453-243cda

In [10]:
unrepaired_bridge

['dd323e98-811b-4e04-9421-608759833360',
 'f6d5befe-88c1-4833-9164-6d7ba2b66ca6',
 '033553d8-100b-4c0f-87a6-de74ffbee25c',
 'b7312b14-76fc-4548-84aa-f331885a41d1',
 '103283a3-548e-4368-a38a-bee6fae481ed',
 'fb960495-a653-47a7-890f-de3a62e2ae2d',
 'cb175bb2-03ef-4814-ab22-ba7fd1dd5c91',
 'b39f942a-555a-4677-9b62-b13441d6f503',
 '66fd8603-c038-4a4d-aec2-667b0ec0e948',
 '72022b56-b4d0-4708-b44a-434d76490362',
 '165914a0-a651-4b13-8dfa-841247935db0',
 '3d84ef35-6170-4386-a18d-43100983345d',
 '2bcc4558-e8be-4cd8-9259-6b4872286783',
 '8e9a135d-339a-4cb0-87e6-a75589a39823',
 'f82b2aa2-eb61-4f8d-b98f-af4eb5cae488',
 'd5ae7be5-b74f-4f56-92bc-8b246a49fd8f',
 '69c93b3e-dc58-412f-bc7e-6c6363b44ccf',
 '66a1b90e-cac6-41ff-91d7-16953d9c3e61',
 '1e0e2f70-69ee-41a1-bb45-a41dc8194921',
 '6d48f606-8e8e-43be-bbc8-7be548f06505',
 '98b8083e-b88f-48a8-88b1-045bc5e26d36',
 '9859f11b-e11a-4fc0-a96e-e5989a6a244c',
 '4c8792c5-70a8-4c1c-b453-243cda28168b',
 '2b8a79d0-a6e0-4608-b93a-f002bc6832a5']

## Calculate Transportation Recovery

In [11]:
nodes_set = InventoryDataset('Data/Links_wgs84_node.shp')
nodes = list(nodes_set.inventory_set)

links_set = InventoryDataset('Data/Links_wgs84.shp')
links = list(links_set.inventory_set)

In [12]:
transportation_recovery = TransportationRecovery(nodes, links, bridges, bridge_damage_value, unrepaired_bridge, ADT)
transportation_recovery.calc_recovery_scenario(pm=1,
                                               ini_num_population=5,
                                               population_size=3,
                                               num_generation=2,
                                               mutation_rate=0.1,
                                               crossover_rate=1.0)

optimal_solution_of_bridge_repair_schedule.csv saved!
overall_transportation_recovery_trajectory.csv saved!


In [13]:
repair_schedule = pd.read_csv('optimal_solution_of_bridge_repair_schedule.csv')
repair_schedule.head()

,Bridge ID,Ending Time
0,b7312b14-76fc-4548-84aa-f331885a41d1,0.6
1,103283a3-548e-4368-a38a-bee6fae481ed,0.6
2,fb960495-a653-47a7-890f-de3a62e2ae2d,0.6
3,cb175bb2-03ef-4814-ab22-ba7fd1dd5c91,1.2
4,66fd8603-c038-4a4d-aec2-667b0ec0e948,0.6


In [14]:
recovery_trajectory = pd.read_csv('overall_transportation_recovery_trajectory.csv')
recovery_trajectory.head()

,Ending Time,Travel Efficiency
0,0.6,0.894795
1,1.2,0.932700
2,1.8,0.974815
3,2.4,0.989937
4,2.5,1.000000
